In [3]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

# Load the DJIA dataset from a CSV file
df = sns.load_dataset('dowjones')

# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort the data by date (if it's not already sorted)
df = df.sort_values(by='Date')

# Create a Dash application
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.H1('DJIA Closing Price Over Time'),
    dcc.Graph(id='djia-graph'),
    dcc.RangeSlider(
        id='date-slider',
        min=df['Date'].min().timestamp(),
        max=df['Date'].max().timestamp(),
        step=1,
        marks={int(i): pd.Timestamp(i, unit='s').strftime('%Y-%m-%d') for i in range(
            int(df['Date'].min().timestamp()), int(df['Date'].max().timestamp()), 3600 * 24 * 30)},
        value=[df['Date'].min().timestamp(), df['Date'].max().timestamp()],
    ),
    html.Div(id='date-label'),
])

# Define callback to update the graph and date labels based on the date range selected
@app.callback(
    [Output('djia-graph', 'figure'), Output('date-label', 'children')],
    Input('date-slider', 'value')
)
def update_graph(selected_dates):
    start_date = pd.Timestamp(selected_dates[0], unit='s')
    end_date = pd.Timestamp(selected_dates[1], unit='s')
    
    filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    fig = px.line(filtered_df, x='Date', y='Price', title='DJIA Closing Price Over Time')
    
    selected_start_date = pd.Timestamp(selected_dates[0], unit='s').strftime('%Y-%m-%d')
    selected_end_date = pd.Timestamp(selected_dates[1], unit='s').strftime('%Y-%m-%d')
    date_range_text = f'Selected Date Range: {selected_start_date} - {selected_end_date}'
    
    return fig, date_range_text

if __name__ == '__main__':
    app.run_server(debug=True)

    
